In [ ]:
#r "nuget:HtmlAgilityPack,1.11.24"
#r "nuget:Microsoft.Data.Analysis,0.4.0"
#r "nuget:CsvHelper,15.0.5"

In [ ]:
using System;
using System.IO;
using System.Linq;
using System.Net;
using System.Text;
using HtmlAgilityPack;
using Microsoft.Data.Analysis;
using XPlot.Plotly;
Encoding.RegisterProvider(CodePagesEncodingProvider.Instance);

In [ ]:
class CandidateResult
{
    public string Name { get; set; }
    public int VotesCount { get; set; }
    public double Percentage { get; set; }
}

class Uik
{
    public string Name { get; set; }
    public List<int> Technical { get; set; }
    public int TotalVoters { get; set; }
    public int TotalVoted { get; set; }
    public int InvalidBallots { get; set; }
    public double Turnout { get; set; }
    public Dictionary<string, CandidateResult> CandidateResults { get; set; }
}

In [ ]:
var url1 = "https://www.cvk.gov.ua/pls/vp2019/wp335pt001f01=719.html";
var url2 = "https://www.cvk.gov.ua/pls/vp2019/wp335pt001f01=720.html";

In [ ]:
var request = WebRequest.CreateHttp(url1);
var webResponse = request.GetResponse();
var doc = new HtmlDocument();
using (var responseStream = webResponse.GetResponseStream())
    doc.Load(responseStream, Encoding.GetEncoding("windows-1251"));

In [ ]:
var links = doc.DocumentNode.SelectNodes("//table/tr/td[2]/a");
var hrefs = links.Select(node => "https://www.cvk.gov.ua/pls/vp2019/" + node.Attributes["href"].Value);

In [ ]:
static List<string> ExtractHeader(HtmlDocument tikDocument)
{
    var header = tikDocument.DocumentNode
        .SelectNodes("//table/thead/tr")
        .First()
        .SelectNodes("th")
        .Select(node => node.InnerHtml.Replace("<br>", " ").Trim())
        .ToList();
    return header;
}

static Uik ParseRow(List<string> tableRow, List<string> candidateNames)
{
    var technical = tableRow.Skip(1).Take(10).Select(int.Parse).ToList();
    var totalVoters = technical[1];
    var totalVoted = technical[8];
    var invalidBallots = technical[9];
    var turnout = (double) totalVoted / totalVoters;

    var results = tableRow.Skip(11).SkipLast(1).Select(int.Parse).ToList();

    var candidateResults = Enumerable.Range(0, candidateNames.Count)
        .Select(i => new CandidateResult()
        {
            Name = candidateNames[i],
            VotesCount = results[i],
            Percentage = (double) results[i] / totalVoted
        })
        .ToDictionary(result => result.Name);

    return new Uik()
    {
        Name = tableRow[0],
        TotalVoters = totalVoters,
        TotalVoted = totalVoted,
        InvalidBallots = invalidBallots,
        Turnout = turnout,
        Technical = technical,
        CandidateResults = candidateResults
    };
}

static List<Uik> ExtractUiks(HtmlDocument tikDocument)
{
    var candidateNames = ExtractHeader(tikDocument).Skip(11).SkipLast(1).ToList();

    return tikDocument.DocumentNode.SelectNodes("//table/tr")
        .Select(row => row.SelectNodes("td")
            .Select(node => node.InnerHtml.Replace("<b>", "")
                .Replace("</b>", ""))
            .ToList())
        .Select(row => ParseRow(row, candidateNames))
        .ToList();
}

static List<T> ParseHtml<T>(string url, Func<HtmlDocument, List<T>> handler)
{
    var tikRequest = WebRequest.CreateHttp(url);
    var tikResponse = tikRequest.GetResponse();
    var tikDocument = new HtmlDocument();
    using (var responseStream = tikResponse.GetResponseStream())
        tikDocument.Load(responseStream, Encoding.GetEncoding("windows-1251"));
    return handler(tikDocument);
}

static List<string> GetHeader(string url) => ParseHtml(url, ExtractHeader);

static List<Uik> GetUiks(string url) => ParseHtml(url, ExtractUiks);

In [ ]:
var rand = new Random();
var header = GetHeader(hrefs.First());
var uiks = hrefs.OrderBy(x => rand.Next()).Take(10)
    .SelectMany(url => GetUiks(url))
    .ToList();

In [ ]:
uiks.First().CandidateResults.Keys

# Рисуем графики

In [ ]:
static Graph.Scatter GetScatter(IEnumerable<Uik> uiks,
    List<string> header,
    string candidateName,
    double markerSize) =>
    new Graph.Scatter()
    {
        x = uiks.Select(uik => uik.Turnout).ToList(),
        y = uiks.Select(uik => uik.CandidateResults[candidateName].Percentage).ToList(),
        mode = "markers",
        marker = new Graph.Marker() { size = markerSize },
        name = candidateName
    };

In [ ]:
var plot = Chart.Plot(new [] {"Порошенко Петро", "Зеленський Володимир"}.Select(candidate => GetScatter(uiks, header, candidate, 1.5)));
var layout = new Layout.Layout
{
    xaxis = new Graph.Xaxis {title = "Явка избирателей"},
    yaxis = new Graph.Yaxis {title = "Процент голосов за кандидата"}
};
plot.WithLayout(layout);
plot.WithTitle("Результат кандидата");
display(plot);